In [1]:
import os

import pandas as pd
import numpy as np

# store elements as dictionary keys and their counts as dictionary values
from collections import Counter

# scikit-learn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline

# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

# Function for creating model pipelines - sklearn
from sklearn.pipeline import make_pipeline

# Function for creating model pipelines - imblearn
from imblearn.pipeline import make_pipeline as imbl_pipe

# Over-sampling using SMOTE
from imblearn.over_sampling import SMOTE

In [2]:
data = pd.read_csv('../Data/data.csv', sep=',')
data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['customer_id'], axis=1)

# Model building

In [4]:
# Object for target variable
y = data.churn

# object for input features
X = data.drop(['churn'], axis=1)

# display shapes of X and y
print(X.shape, y.shape)

(10000, 10) (10000,)


In [5]:
# List numerical features
num_columns = X.select_dtypes(include='number').columns.tolist()
num_columns

['credit_score',
 'age',
 'tenure',
 'balance',
 'products_number',
 'credit_card',
 'active_member',
 'estimated_salary']

In [6]:
# List categorical features
cat_columns = X.select_dtypes(include='object').columns.tolist()
cat_columns

['country', 'gender']

In [7]:
random_state = 10

# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=random_state,
                                                    stratify=data.churn)

# Print number of observations in X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test), len(y_train), len(y_test))

7000 3000 7000 3000


In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 8061 to 4741
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   credit_score      7000 non-null   int64  
 1   country           7000 non-null   object 
 2   gender            7000 non-null   object 
 3   age               7000 non-null   int64  
 4   tenure            7000 non-null   int64  
 5   balance           7000 non-null   float64
 6   products_number   7000 non-null   int64  
 7   credit_card       7000 non-null   int64  
 8   active_member     7000 non-null   int64  
 9   estimated_salary  7000 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 601.6+ KB


# Scaling - Encoding the data

In [9]:
num_features = [] 

for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features) 

[0, 3, 4, 5, 6, 7, 8, 9]


In [10]:
cat_features = []

for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
cat_features

[1, 2]

In [11]:
# Define the column transformer
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(sparse_output=False), cat_features)  
)


print(preprocess)

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [0, 3, 4, 5, 6, 7, 8, 9]),
                                ('onehotencoder',
                                 OneHotEncoder(sparse_output=False), [1, 2])])


## Build Model Pipeline with SMOTE

* We are going to use the Pipeline from the imblearn package in place of scikit-learn Pipeline.

* It takes care automatically to re-sample when called fit() on the pipeline, and does not re-sample test data (when called transform() or predict()).

In [12]:
# Import classifier
from sklearn.svm import SVC 

# Define model with pipeline
model = imbl_pipe(preprocess,
                  SMOTE(sampling_strategy='auto', random_state=random_state),
                  SVC(random_state=random_state))

model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [0, 3, 4, 5, 6, 7, 8, 9]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  [1, 2])])),
                ('smote', SMOTE(random_state=10)),
                ('svc', SVC(random_state=10))])

In [13]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV

param_grid = {'svc__kernel' : ['linear', 'rbf', 'poly', 'sigmoid'],
              'svc__C': [0.0005,0.001, 0.01, 0.1, 0.5],
              'svc__gamma': [5, 1, 0.1, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3, cv= 5, n_jobs=4, scoring='accuracy')

In [14]:
X_train = X_train.values
X_test = X_test.values

In [15]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('minmaxscaler',
                                                                         MinMaxScaler(),
                                                                         [0, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8,
                                                                          9]),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(sparse_output=False),
                                                                         [1,
                                                                          2])])),
                                       ('smote', SMOTE(random_state=10)),
                                       ('svc', SVC(random_state=10))]),
             n_jobs=4,
             param_grid={'svc__C': [0.0005, 0.001, 0.01, 0.1, 0.5],
                         'svc__gamma': [5, 1, 0.1, 0.01],
                         'svc__kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='accuracy', verbose=3)

In [16]:
print(grid.best_params_)

{'svc__C': 0.5, 'svc__gamma': 5, 'svc__kernel': 'poly'}


In [17]:
print(grid.best_score_)

0.7864285714285714


In [18]:
print(f"Training Data Score: {grid.score(X_train, y_train)}")
print(f"Testing Data Score: {grid.score(X_test, y_test)}")

Training Data Score: 0.811
Testing Data Score: 0.7756666666666666


In [19]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)
predictions

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [20]:
# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, predictions)
print(cm)

[[1872  517]
 [ 156  455]]


In [21]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.92      0.78      0.85      2389
           1       0.47      0.74      0.57       611

    accuracy                           0.78      3000
   macro avg       0.70      0.76      0.71      3000
weighted avg       0.83      0.78      0.79      3000



In [22]:
predictions

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [23]:
pred = grid.predict(X_test[:10])

In [24]:
print(f"Predicted classes: {pred}")
print(f"Actual Labels: {list(y_test[:1])}")

Predicted classes: [1 0 0 0 1 1 0 0 0 1]
Actual Labels: [1]


In [25]:
predictions =grid.predict(X_test[:15])
print(f"First 15 Predictions:   {predictions[:15]}")
print(f"First 15 Actual labels: {y_test[:15].tolist()}")

First 15 Predictions:   [1 0 0 0 1 1 0 0 0 1 1 0 1 0 0]
First 15 Actual labels: [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0]


* save the model

In [26]:
import joblib

#  saving grid model
filename = '../models/SVM_model.sav'
joblib.dump(grid, filename)

['../models/SVM_model.sav']

In [27]:
# load the model
svm_model = joblib.load(filename)
print(svm_model.score(X_test, y_test))

0.7756666666666666


 * Predict class for new data

In [28]:
# Let's use the first 4 X_test record as new data
X_test[:4]

array([[638, 'France', 'Male', 36, 6, 188455.19, 1, 0, 0, 47031.4],
       [640, 'Spain', 'Male', 29, 5, 197200.04, 2, 1, 0, 141453.62],
       [826, 'France', 'Female', 30, 5, 0.0, 2, 0, 1, 157397.57],
       [717, 'Spain', 'Female', 36, 2, 164557.95, 1, 0, 1, 82336.73]],
      dtype=object)

In [29]:
pred_new = grid.predict(X_test[:4])

In [30]:
print(f"Predicted classes: {pred_new}")
print(f"Actual Labels: {list(y_test[:4])}")

Predicted classes: [1 0 0 0]
Actual Labels: [1, 0, 0, 0]
